In [1]:
# %pip install plotly
# %pip install nbformat
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

os.getcwd()

'/Users/aneeshussain/Code/diss/analysis'

In [2]:
# Get the current directory
current_dir = os.getcwd()

# Change directory to the parent directory (diss)
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
os.chdir(parent_dir)

# Append the parent directory to the system path (optional, if you want to add it to the import path)
sys.path.append(parent_dir)

# Print the new current working directory to confirm the change
print(f"Changed directory to: {os.getcwd()}")

Changed directory to: /Users/aneeshussain/Code/diss


In [3]:
filepaths = {
    'binance': './data/processed/binance.csv',
    'okx': './data/processed/okx.csv',
    'bybit': './data/processed/bybit.csv'
}

# List of specified pairs
specified_pairs = ['BTCUSDCM', 'BTCUSDT', 'ETHUSDCM', 'ETHUSDT']

# Initialize a dictionary to store counts for each exchange
exchange_funding_stats = {}

# Loop through each exchange
for exchange, filepath in filepaths.items():
    # Load the data for the current exchange
    df = pd.read_csv(filepath)

    # Convert 'funding rate' to float
    df['funding rate'] = pd.to_numeric(df['funding rate'], errors='coerce')

    # Filter the DataFrame to only include the specified pairs
    filtered_df = df[df['pair'].isin(specified_pairs)]

    # Initialize dictionaries to store counts for this exchange
    positive_funding_counts = {}
    total_funding_counts = {}

    # Iterate through each specified pair
    for pair in specified_pairs:
        # Filter rows for the current pair
        pair_df = filtered_df[filtered_df['pair'] == pair]

        # Count positive funding rates
        positive_count = (pair_df['funding rate'] > 0).sum()
        total_count = len(pair_df)

        # Store the counts in the dictionaries
        positive_funding_counts[pair] = positive_count
        total_funding_counts[pair] = total_count

    # Create a DataFrame for this exchange to display the counts
    funding_stats = pd.DataFrame({
        'positive_count': positive_funding_counts,
        'total_count': total_funding_counts
    })

    # Calculate the percentage of positive funding rates
    funding_stats['positive_percentage'] = (funding_stats['positive_count'] / funding_stats['total_count']) * 100

    # Store the result for this exchange
    exchange_funding_stats[exchange] = funding_stats

# Display the results for each exchange
for exchange, stats in exchange_funding_stats.items():
    print(f"{exchange} Funding stats")
    print(stats)
    print("\n")

binance Funding stats
          positive_count  total_count  positive_percentage
BTCUSDCM            1435         1642            87.393423
BTCUSDT             1517         1642            92.387333
ETHUSDCM            1467         1642            89.342266
ETHUSDT             1539         1642            93.727162


okx Funding stats
          positive_count  total_count  positive_percentage
BTCUSDCM            1132         1642            68.940317
BTCUSDT             1203         1642            73.264312
ETHUSDCM            1163         1642            70.828258
ETHUSDT             1226         1642            74.665043


bybit Funding stats
          positive_count  total_count  positive_percentage
BTCUSDCM            1472         1642            89.646772
BTCUSDT             1494         1642            90.986602
ETHUSDCM            1481         1642            90.194884
ETHUSDT             1474         1642            89.768575




In [4]:
df = pd.read_csv('./results/hold/funding_log.csv')
print(f"Hold average funding rate: {df['funding payment'].sum()}")

df = pd.read_csv('./results/simple_threshold/funding_log.csv')
print(f"Simple average funding rate: {df['funding payment'].sum()}")

df = pd.read_csv('./results/simple_reinvest/funding_log.csv')
print(f"Simple Reinvest average funding rate: {df['funding payment'].sum()}")

df = pd.read_csv('./results/complex_threshold/funding_log.csv')
print(f"Complex average funding rate: {df['funding payment'].sum()}")

df = pd.read_csv('./results/complex_reinvest/funding_log.csv')
print(f"Complex Reinvest average funding rate: {df['funding payment'].sum()}")

Hold average funding rate: 3581973.695275749
Simple average funding rate: 1254712.4667579024
Simple Reinvest average funding rate: 1290910.318003559
Complex average funding rate: 1278043.596267812
Complex Reinvest average funding rate: 1317241.4917293699


In [8]:
df = pd.read_csv('./data/processed/okx_eth.csv')
df['time'] = pd.to_datetime(df['time'])

perpetual_df = df[df['contract'] == 'perpetual']

perpetual_df['date'] = perpetual_df['time'].dt.date

daily_funding_rate = perpetual_df.groupby('date')['funding rate'].sum()

daily_mean_funding_rate = daily_funding_rate.mean()
annualised_avg_funding_rate = (1 + daily_mean_funding_rate) ** 365 - 1
print(daily_mean_funding_rate)
print(annualised_avg_funding_rate * 100)

0.0005059947208201541
20.278699915911247


/var/folders/mf/b67_blxs1rj380h3rh2kgtlh0000gn/T/ipykernel_50091/303903876.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perpetual_df['date'] = perpetual_df['time'].dt.date


In [6]:
file_paths = {
    'Hold': './results/hold/yield.csv',
    'Simple': './results/simple_threshold/yield.csv',
    'Simple Reinvest': './results/simple_reinvest/yield.csv',
    'Complex': './results/complex_threshold/yield.csv',
    'Complex Reinvest': './results/complex_reinvest/yield.csv'
}

mean_values_dict = {}

for strategy, file_path in file_paths.items():
    df = pd.read_csv(file_path)

    df_no_first_column = df.iloc[:, 1:]
  
    mean_values = df_no_first_column.mean()
    
    # annualised_mean_values = (1 + mean_values) ** 365 - 1
    # mean_values_dict[strategy] = annualised_mean_values
    mean_values_dict[strategy] = mean_values

mean_table = pd.DataFrame(mean_values_dict).transpose()

mean_table = mean_table.round(3)

output_file = './results/compare/mean_yield_table.csv'
mean_table.to_csv(output_file)

print(f"Table saved to {output_file}")

Table saved to ./results/compare/mean_yield_table.csv


In [12]:
df = pd.read_csv('./results/hold/yield.csv')

df_no_first_column = df.iloc[:, 1:]

mean_values = df_no_first_column.mean()

print(mean_values)

# annualised_mean_values = (1 + mean_values) ** 365 - 1

# print(annualised_mean_values)

binance_btcusd_yield    0.570631
binance_btccm_yield     0.651782
binance_ethusd_yield    0.373980
binance_ethcm_yield     0.385915
bybit_btcusd_yield      0.671859
bybit_btccm_yield       0.859515
bybit_ethusd_yield      0.343129
bybit_ethcm_yield       0.359032
okx_btcusd_yield        0.625258
okx_btccm_yield         0.587621
okx_ethusd_yield        0.344771
okx_ethcm_yield         0.381536
binance_btc_yield       1.222412
binance_eth_yield       0.759894
bybit_btc_yield         1.531374
bybit_eth_yield         0.702161
okx_btc_yield           1.212879
okx_eth_yield           0.726307
binance_total_yield     1.982307
bybit_total_yield       2.233535
okx_total_yield         1.939186
btc_total_yield         3.966666
eth_total_yield         2.188362
total_yield             6.155028
dtype: float64
